<a href="https://colab.research.google.com/github/ffelfis/OrgaDatosTPs/blob/main/TP2/resources/OPT_Bayesian_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lectura de datos de Google Drive

In [ ]:
# Lectura de Dataset desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Para importar funciones customizadas

Hay que especificar la ruta de donde se encuentra el módulo (archivo `.py`) para buscar las funciones.

La lectura puede ser muy celosa: las indentaciones son de 4 espacios no tabulaciones.

https://colab.research.google.com/drive/1uvHuizCBqFgvbCwEhK7FvU8JW0AfxgJw

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/75.06 - Organización de Datos/TP2/resources')

# Instalación de librerías

Hay que instalar `scikit-optimize`.

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 101 kB 5.6 MB/s 


# Carga de librerías y directorios

In [ ]:
import pandas as pd
import numpy as np

# Métrica
from sklearn.metrics import f1_score

# Clasificador
from sklearn.ensemble import RandomForestClassifier

# Paquete de Grid Search Cross Validation
from sklearn.model_selection import GridSearchCV

# Para el cambio de tipo de datos: para reducir el uso de memoria
from utilidades import cambio_tipos

# Para la división en k-folds
from sklearn.model_selection import StratifiedKFold

# Para búsqueda Bayesiana
from functools import partial
# Para función a minimizar
from skopt import space
from skopt import gp_minimize

# Rutas de los archivos a usar

In [ ]:
# Ruta train_values.csv
dir_values = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_values.csv'
# Ruta train_labels.csv
dir_labels = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_labels.csv'
# Ruta de Binary Encodings para train_values.csv
dir_Binary = '/content/drive/My Drive/75.06 - Organización de Datos/TP2/resources'

---
#Entrenamiento
---
### Carga de train

In [ ]:
columnas = ['building_id',
 'geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_superstructure_other',
 'count_families',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other']

# Carga de train_values.csv
train = pd.read_csv(dir_values, usecols=columnas)

### Cambio de tipos de datos

In [ ]:
train = cambio_tipos(train)

### Carga de columnas codificadas: Binary Encoding

In [ ]:
# 28 columnas más.
train = train.join(pd.read_csv(dir_Binary+f'/BE_train.csv', dtype='uint8'))

X = train.values

### Carga de labels

In [ ]:
# Carga de train_labels.csv
labels = pd.read_csv(dir_labels, usecols=['damage_grade'], dtype='uint8')

y = labels.damage_grade.values

### Instancia del clasificador.

In [ ]:
rfc = RandomForestClassifier()

# Bayesian Search

Hay que crear una función de optimización.

https://stackoverflow.com/questions/55667169/key-error-none-of-int64index-dtype-int64-are-in-the-columns 

In [ ]:
# params: lista de valores de parámetros (value)
# param_names: lista de nombres de los parámetros (key)
# x: features
# y: target

def optimize(params, param_names, x, y):
  # Se crea un diccionario con los parámetros y sus nombres
  params = dict(zip(param_names, params))
  # Se instancia el modelo con el diccionario de parámetros
  model = RandomForestClassifier(**params)
  # Hay que realizar el k-folding
  kf = StratifiedKFold(n_splits=5)
  # Lista de scores
  scores = []
  # Se dividen los datos
  for index in kf.split(X=x, y=y):
    train_index, test_index = index[0], index[1]
    X_train = x[train_index]
    y_train = y[train_index]
    X_test = x[test_index]
    y_test = y[test_index]
    # Entrenamiento del k-fold
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    fold_score = f1_score(y_test, prediction, average='micro')
    # Se agrega el score a la lista
    scores.append(fold_score)
  # Ahora hay que devolver la función para minimizar
  return -1.0 * np.mean(scores)

### Espacio de parámetros para realizar la búsqueda

In [ ]:
param_space = [
  space.Integer(30, 50, name='max_depth'),
  space.Integer(20, 50, name='n_estimators'),
  space.Categorical(['gini', 'entropy'], name='criterion'),
  space.Real(0.05, 1, prior='uniform', name='max_features')
]

Se definen los nombres de los parámetros.

In [ ]:
param_names = [
  'max_depth',
  'n_estimators', 
  'criterion',
  'max_features'
]

Función para optimizar (partial: se pasan algunos parámetros y algunos de ellos se pasan muchas veces).

In [ ]:
optimization_function = partial(optimize, 
                                param_names = param_names, 
                                x=X, 
                                y=y)

Hay que ejecutar la función `gp_minimize`.

In [ ]:
result = gp_minimize(optimization_function, 
                     dimensions=param_space, 
                     n_calls=15, 
                     n_random_starts=10, 
                     verbose=10)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 324.9155
Function value obtained: -0.7268
Current minimum: -0.7268
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 488.2390
Function value obtained: -0.7281
Current minimum: -0.7281
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 224.8897
Function value obtained: -0.7205
Current minimum: -0.7281
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 277.5184
Function value obtained: -0.7239
Current minimum: -0.7281
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 291.5619
Function value obtained: -0.7257
Current minimum: -0.7281
Iteration No: 6

Se muestra los parámetros.

In [ ]:
type(result)

scipy.optimize.optimize.OptimizeResult

In [ ]:
print('Mejores parámetros del espacio de búsqueda:')
print(dict(zip(param_names,result.x)))

Mejores parámetros del espacio de búsqueda:
{'max_depth': 30, 'n_estimators': 50, 'criterion': 'entropy', 'max_features': 1.0}
